## Load Base Stable Diffusion Model

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline
from pruna.algorithms.smasher_config import AutoSmasherConfig
from pruna.smash import smash
import torch
import requests
from PIL import Image
from io import BytesIO

import cv2
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler, StableDiffusionImg2ImgPipeline
import torch
import numpy as np
from diffusers.utils import load_image

image = Image.open(
            BytesIO(
                requests.get(
                    "https://huggingface.co/lllyasviel/sd-controlnet-canny/resolve/main/images/bird_canny.png"
                ).content
            )
        )

controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

pipe.to('cuda')


## Smash it!

In [ ]:
from pruna.algorithms.smasher_config import SmasherConfig

smasher_config = SmasherConfig()
smasher_config['compiler'] = 'diffusers'
smasher_config['max_batch_size'] = 1
smasher_config['image_height'] = 512
smasher_config['image_width'] = 512
smasher_config['version'] = '1.5'
smasher_config['static_shape'] = False
smasher_config['controlnet'] = True

In [ ]:
smashed_model = smash(
        model=pipe,
        data_module="img2img",
        api_key='your-api-key',
        model_config=None,
        smasher_config=smasher_config,
        device='cuda',
    )

In [ ]:
prompt = "Yellow Bird"

In [ ]:
pipe

## Base Model Generation

In [ ]:
%%time
pipe(prompt=prompt, image=image).images[0].show()

## Smashed Model Generation

In [ ]:
%%time
smashed_model(prompt, init_image=image, image_height=768, image_width=512)[0]